# TRITON-SWMM Toolkit Quickstart

This notebook demonstrates the high-level `Toolkit` API for running TRITON-SWMM workflows.

The Toolkit provides a simplified, notebook-friendly interface that wraps the underlying Analysis class with sensible defaults and clear method names.

## 1. Setup and Configuration

First, import the Toolkit and create an instance from your configuration files.

In [ ]:
from TRITON_SWMM_toolkit import Toolkit
from pathlib import Path

# Define paths to your configuration files
# Replace these with your actual config file paths
system_config = Path("../test_data/norfolk_coastal_flooding/template_system_config.yaml")
analysis_config = Path("../test_data/norfolk_coastal_flooding/template_analysis_config.yaml")

# Initialize the toolkit
# This loads configs, processes system inputs, and compiles executables
tk = Toolkit.from_configs(
    system_config=system_config,
    analysis_config=analysis_config,
    validate=True  # Run preflight validation (recommended)
)

print(tk)

## 2. Inspect Analysis Configuration

Check basic properties of your analysis setup.

In [ ]:
print(f"Analysis directory: {tk.analysis_dir}")
print(f"Total simulations: {tk.n_simulations}")
print(f"Analysis ID: {tk.analysis.cfg_analysis.analysis_id}")

## 3. Run a Fresh Workflow

Execute the complete workflow from scratch, overwriting any existing outputs.

In [ ]:
# Run workflow in "fresh" mode (from scratch, overwrite everything)
result = tk.run(mode="fresh", verbose=True)

# Check result
if result.success:
    print(f"\n✓ Workflow completed successfully!")
    print(f"  Runtime: {result.execution_time:.1f} seconds")
    print(f"  Events processed: {result.events_processed}")
    print(f"  Phases completed: {result.phases_completed}")
else:
    print(f"\n✗ Workflow failed: {result.message}")

## 4. Check Workflow Status

Get a detailed status report showing progress of each phase.

In [ ]:
# Get current status
status = tk.get_status()

# Print formatted status report
print(status)

# Access specific status details
print(f"\nSimulations: {status.simulations_completed}/{status.total_simulations} complete")
print(f"Failed: {status.simulations_failed}")
print(f"Recommended mode: {status.recommended_mode}")

## 5. Resume an Interrupted Workflow

If a workflow was interrupted or you need to process additional data, use resume mode.

In [ ]:
# Check status first
status = tk.get_status()
print(f"Current progress: {status.simulations_completed}/{status.total_simulations}")
print(f"Recommendation: {status.recommendation}")

# Resume from last checkpoint
result = tk.run(mode="resume", verbose=True)

if result.success:
    print(f"\n✓ Workflow resumed successfully!")
    print(f"  Additional events processed: {len(result.events_processed)}")

## 6. Run Specific Events Only

Process a subset of events instead of the entire analysis.

In [ ]:
# Run only events 0-4
result = tk.run(
    mode="resume",
    events=list(range(5)),  # Events 0, 1, 2, 3, 4
    verbose=True
)

print(f"Processed {len(result.events_processed)} events: {result.events_processed}")

## 7. Run Specific Workflow Phases

Execute only certain phases of the workflow (e.g., skip setup/preparation).

In [ ]:
# Run only the simulation phase (assumes setup/preparation already complete)
result = tk.run(
    mode="resume",
    phases=["simulation"],
    verbose=True
)

print(f"Phases completed: {result.phases_completed}")

## 8. Dry Run (Preview Without Executing)

See what would be executed without actually running anything.

In [ ]:
# Preview workflow plan
result = tk.run(mode="fresh", dry_run=True, verbose=True)

print(f"\nDry run message:\n{result.message}")

## 9. Overwrite Existing Results

Rerun scenarios even if completion logs indicate success.

In [ ]:
# Overwrite mode: rerun existing scenarios without full reset
result = tk.run(mode="overwrite", verbose=True)

if result.success:
    print(f"✓ Overwrote {len(result.events_processed)} existing scenarios")

## 10. Access Underlying Analysis Object

For advanced use cases, access the full Analysis API directly.

In [ ]:
# Access the underlying Analysis object for fine-grained control
analysis = tk.analysis

# Access analysis properties
print(f"Backend: {analysis.backend}")
print(f"In SLURM: {analysis.in_slurm}")
print(f"Execution strategy: {type(analysis._execution_strategy).__name__}")

# Access scenario objects
print(f"\nScenario run objects: {len(analysis._sim_run_objects)}")

# Access status dataframe
print(f"\nStatus DataFrame shape: {analysis.df_status.shape}")
print(analysis.df_status.head())

## Summary

The `Toolkit` class provides:

1. **Simple initialization**: `Toolkit.from_configs(system_cfg, analysis_cfg)`
2. **Three execution modes**:
   - `"fresh"`: Start from scratch, overwrite everything
   - `"resume"`: Continue from last checkpoint (default)
   - `"overwrite"`: Rerun existing scenarios
3. **Status reporting**: `tk.get_status()` with recommendations
4. **Selective execution**: Run specific events or phases
5. **Dry run**: Preview workflow without executing
6. **Direct access**: Use `tk.analysis` for advanced features

For more control, you can use the lower-level `Analysis` class directly. See the toolkit source code and documentation for details.